In [95]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import plotly.offline as pyo
pyo.init_notebook_mode()

In [96]:
directory_data = "../data"

athletes = pd.read_csv(f"{directory_data}/athlete_events.csv")
regions = pd.read_csv(f"{directory_data}/noc_regions.csv")

In [97]:
athletes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [98]:
merged = pd.merge(athletes, regions, on="NOC", how="outer")

In [99]:
medals_total = (
    merged
    .dropna(subset=["Medal"])
    .groupby(["Year", "Medal"], as_index=False)
    .agg(MedalCount=("Medal", "count"))
)


In [100]:
medals_total["Medal"] = pd.Categorical(
    medals_total["Medal"],
    categories=["Gold", "Silver", "Bronze"],
    ordered=True
)

In [101]:
medals_total = medals_total.sort_values(["Year", "Medal"])

In [102]:
medals_total.head()

,Year,Medal,MedalCount
1,1896.0,Gold,62
2,1896.0,Silver,43
0,1896.0,Bronze,38
4,1900.0,Gold,201
5,1900.0,Silver,228


In [103]:
df = px.data.medals_long()

fig = px.bar(df, x="medal", y="count", color="nation", text="nation")
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271117 entries, 0 to 271116
Data columns (total 17 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  float64
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271117 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  float64
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
 15  region  270747 non-null  object 
 16  notes   5039 non-null    object 
dtypes: float64(5), object(12)
memory usage: 35.2+ MB


In [ ]:
merged

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,502.0,Ahmad Shah Abouwi,M,NaN,NaN,NaN,Afghanistan,AFG,1956 Summer,1956.0,Summer,Melbourne,Hockey,Hockey Men's Hockey,NaN,Afghanistan,NaN
1,1076.0,Jammal-ud-Din Affendi,M,28.0,NaN,NaN,Afghanistan,AFG,1936 Summer,1936.0,Summer,Berlin,Hockey,Hockey Men's Hockey,NaN,Afghanistan,NaN
2,1101.0,Mohammad Anwar Afzal,M,NaN,NaN,NaN,Afghanistan,AFG,1948 Summer,1948.0,Summer,London,Football,Football Men's Football,NaN,Afghanistan,NaN
3,1745.0,Mohammad Aktar,M,17.0,156.0,48.0,Afghanistan,AFG,1980 Summer,1980.0,Summer,Moskva,Wrestling,"Wrestling Men's Light-Flyweight, Freestyle",NaN,Afghanistan,NaN
4,4628.0,Mohammad Daoud Anwary,M,22.0,NaN,NaN,Afghanistan,AFG,1964 Summer,1964.0,Summer,Tokyo,Wrestling,"Wrestling Men's Bantamweight, Freestyle",NaN,Afghanistan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271112,130832.0,Hillary Wilson,F,15.0,157.0,65.0,Zimbabwe,ZIM,1960 Summer,1960.0,Summer,Roma,Swimming,Swimming Women's 100 metres Butterfly,NaN,Zimbabwe,NaN
271113,130832.0,Hillary Wilson,F,15.0,157.0,65.0,Zimbabwe,ZIM,1960 Summer,1960.0,Summer,Roma,Swimming,Swimming Women's 4 x 100 metres Medley Relay,NaN,Zimbabwe,NaN
271114,130880.0,Peter Arthur Wilson,M,20.0,185.0,80.0,Zimbabwe,ZIM,1980 Summer,1980.0,Summer,Moskva,Sailing,Sailing Mixed One Person Dinghy,NaN,Zimbabwe,NaN
271115,131478.0,"Jennifer ""Jenny"" Wood",F,16.0,172.0,64.0,Zimbabwe,ZIM,1964 Summer,1964.0,Summer,Tokyo,Swimming,Swimming Women's 100 metres Butterfly,NaN,Zimbabwe,NaN


In [ ]:
merged_ANZ = merged[merged["NOC"] == "ANZ"]

In [ ]:
merged_ANZ

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
1395,1537.0,William Victor Aitken,M,19.0,NaN,NaN,Australasia,ANZ,1908 Summer,1908.0,Summer,London,Athletics,Athletics Men's Marathon,NaN,Australia,Australasia
1396,7037.0,"Reginald Leslie ""Snowy"" Baker",M,24.0,178.0,NaN,Australasia,ANZ,1908 Summer,1908.0,Summer,London,Diving,Diving Men's Springboard,NaN,Australia,Australasia
1397,7037.0,"Reginald Leslie ""Snowy"" Baker",M,24.0,178.0,NaN,Australasia,ANZ,1908 Summer,1908.0,Summer,London,Boxing,Boxing Men's Middleweight,Silver,Australia,Australasia
1398,7037.0,"Reginald Leslie ""Snowy"" Baker",M,24.0,178.0,NaN,Australasia,ANZ,1908 Summer,1908.0,Summer,London,Swimming,Swimming Men's 4 x 200 metres Freestyle Relay,NaN,Australia,Australasia
1399,7980.0,John Thomas Barnett,M,28.0,NaN,NaN,Australasia,ANZ,1908 Summer,1908.0,Summer,London,Rugby,Rugby Men's Rugby,Gold,Australia,Australasia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,118661.0,"Theodore ""Theo"" Tartakover",M,31.0,NaN,NaN,Australasia,ANZ,1912 Summer,1912.0,Summer,Stockholm,Swimming,Swimming Men's 400 metres Freestyle,NaN,Australia,Australasia
1477,128071.0,Robert George Krulock Waley,M,22.0,NaN,NaN,Sydney Rowing Club,ANZ,1912 Summer,1912.0,Summer,Stockholm,Rowing,Rowing Men's Coxed Eights,NaN,Australia,Australasia
1478,128715.0,Hugh Kingsley Ward,M,22.0,NaN,NaN,Sydney Rowing Club,ANZ,1912 Summer,1912.0,Summer,Stockholm,Rowing,Rowing Men's Coxed Eights,NaN,Australia,Australasia
1479,130415.0,"Anthony Frederick ""Tony"" Wilding",M,28.0,188.0,84.0,Australasia,ANZ,1912 Summer,1912.0,Summer,Stockholm,Tennis,"Tennis Men's Singles, Covered Courts",Bronze,Australia,Australasia


### Checking if New Zealand i specified in region for ANZ

In [ ]:
merged_ANZ['region'].isin(["New Zealand"]).any()


np.False_